In [1]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse, SemanticEmbeddingRequest, SemanticEmbeddingResponse, SemanticRepresentation
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

client = Client(token=os.getenv("AA_TOKEN"))

### Step 1: Let's do a really simple QA prompt
In this step we will ask luminous a question from the manual. Let's see how it answers.
Write a completionrequest and make luminous answer it. Don't forget the stop sequences.
You can find the documentation for the completionrequest here: https://docs.aleph-alpha.com/docs/tasks/complete/

In [3]:
prompt = "Q: Is the world flat?\nA:"

# TODO: Write the completion request
request = CompletionRequest(prompt=Prompt.from_text(prompt), maximum_tokens=32, stop_sequences=["\n"])

# TODO: Call the client to run the completionrequest
response = client.complete(request, model="luminous-extended")
print(response.completions[0].completion)

 No, the world is not flat.


#### Step 2: Let's do a QA prompt with a context
Obviously, we can't expect luminous to answer the question if we don't give it any context. Let's see how it answers when we give it some context.
Write the new prompt and make luminous answer it. Don't forget the stop sequences.

In [8]:
context = """The robot has protective and emergency stop functions (stop category 0 or 1, in accordance with IEC 60204-1).

| Stop category 0 | As defined in IEC 60204-1, stopping by immediate removal of power to the machine actuators. |  
|---|---|  
| Stop category 1 | As defined in IEC 60204-1, a controlled stop with power avail- able to the machine actuators to achieve the stop and then re- moval of power when the stop is achieved. |  """

# TODO write a prompt that makes luminous answer the question based on the context
prompt = f"""### Instructions: Answer the question based on the provided Context. If the answer is not in the text return "not answerable".
### Input: {context}
### Question: How is Stop Category 0 defined?
Short Answer:"""

request = CompletionRequest(prompt=Prompt.from_text(prompt), maximum_tokens=32, stop_sequences=["###"])

response = client.complete(request, model="luminous-base-control-beta")
print(response.completions[0].completion)



 Stop Category 0 is defined as stopping by immediate removal of power to the machine actuators.


#### Step 3: Let's use our search we coded in the previous notebook to find the relevant context for the question and then use that context to answer the question
This step is a bit long, but it is a good exercise to see how we can use the search we coded in the previous notebook to find the relevant context for the question and then use that context to answer the question.

In [20]:
texts = [
    "Pinocchio was not a boy, but a wooden puppet. He was made by a carpenter named Geppetto. He was a very naughty puppet. He was always getting into trouble. He was always lying.",
    "Most commonly associated with the polar regions, permafrost is soil and rocky material that stays frozen continuously for at least two years. Normally it lies beneath an active layer that melts and freezes depending on the season. Less well known is that permafrost can also be found on steep mountain walls.",
    "Toheroa are a clam that grow as large as a human hand and burrow in intertidal sands on just a handful of epic surf-swept beaches found mainly on the west coast of New Zealand's North Island, but also in isolated colonies at places like Oreti, a beach at the nation's southern tip.",
    "On the neighbourhood's southern edge, cutting through Queens like a backbone, is Roosevelt Avenue. Here, conversations don't stop when the 7 train rattles overhead, they just get louder. Phone repair shops run by Tibetans with makeshift shrines displayed between plastic iPhone covers abut Latin American bakeries churning out pillowy almojábanas (Colombian cheese bread) and crispy empanadas."
]
        
question = "Who made Pinocchio?"

#TODO: embed the contexts

embedded_texts = []
for text in texts:
    embedded_texts.append(client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document, compress_to_size=128), model="luminous-base").embedding)

# TODO: embed the question
embedded_question = client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(question), representation=SemanticRepresentation.Query, compress_to_size=128), model="luminous-base").embedding

# TODO: calculate the cosine similarity between the question and the contexts
similarities = []
for embedded_text in embedded_texts:
    similarities.append(1 - spatial.distance.cosine(embedded_text, embedded_question))

# Get the text in the most similar context
selected_context = texts[np.argmax(similarities)]

# TODO: write a prompt that makes luminous answer the question based on the context
prompt = f"""### Instructions: Answer the question briefly based on the provided Input. If the answer is not in the text return "not answerable".
### Input: {selected_context}
### Question: {question}
### Response:"""

# TODO: write the CompletionRequest
request = CompletionRequest(prompt=Prompt.from_text(prompt), maximum_tokens=64, stop_sequences=["###"])

response = client.complete(request, model="luminous-base-control-beta")

print(response.completions[0].completion)
print(selected_context)


 Pinocchio was made by a carpenter named Geppetto.
Pinocchio was not a boy, but a wooden puppet. He was made by a carpenter named Geppetto. He was a very naughty puppet. He was always getting into trouble. He was always lying.


### Let's try this with a more complex setup and a vector database

In [53]:
# First we spin up the Qdrant server
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, Batch

q_client = QdrantClient(path="db")

q_client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=128, distance=Distance.COSINE),
)



True

In [22]:
# next we load our data from data.md

with open("data.md", "r") as f:
    data = f.read()
    
# Lets split the data by headings (###)
data = data.split("###")

# remove the first element as it is empty
data = data[1:]

In [52]:
# Let's take a look at the data
data

[' Highlights\n\n- Provides high performance and scalability\n\n- Develops and deploys new optimization-based models\n\n- Makes a difference with real- world applications\n\n- Available for testing via the free Community Edition\n\n- Enables flexible deployment of optimization models on premises and on cloud\n\n',
 ' Overview\n\nDecision optimization is a mathematical technique used to help  \nmake complex business decisions that have the potential for business disruption and involve many large data sources, multiple trade-  \noff possibilities and complex constraints. This advanced analytics technique is often used for planning, scheduling and pricing, as well as other business applications. Organizations across the world have seen business value and high ROI from optimization, as demonstrated through successful projects in a wide range of industries.\n\nIBM Decision Optimization products enable business decision-  \nmaking processes such as operational, tactical and strategic plannin

In [34]:
# Let's create embeddings for each of the texts and store them in a list
embeddings = []
for text in data:
    # TODO: embed the texts
    embeddings.append(client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document, compress_to_size=128), model="luminous-base").embedding)

In [37]:
# now we can upsert the data into Qdrant
ids = list(range(len(data)))
payloads = [{"text": text} for text in data]

q_client.upsert(
     collection_name="test_collection",
     points=Batch(
     ids=ids,
     payloads=payloads,
     vectors=embeddings
     )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [49]:

# TODO write a function that takes a question and returns an answer by searching in the Qdrant database
def search_and_answer(question):
    # TODO First we embed the question
    embedded_question = client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(question), representation=SemanticRepresentation.Query, compress_to_size=128), model="luminous-base").embedding
    
    # Then we search for the most similar text
    search_result = q_client.search(
        collection_name="test_collection",
        query_vector=embedded_question,
        filter=None,
        top=1
    )
    
    # Then we get the text from the search result
    text = search_result[0].payload["text"]
    
    # TODO Finally we ask luminous to answer the question based on the text
    prompt = f"""### Instructions: Answer the question briefly based on the provided Input. If the answer is not in the text return "not answerable".
    
    ### Input: {text}
    
    ### Question: {question}
    
    ### Response:"""
    
    # TODO write the CompletionRequest
    request = CompletionRequest(prompt=Prompt.from_text(prompt), maximum_tokens=64, stop_sequences=["###"])
    
    # TODO get the response from luminous
    response = client.complete(request, model="luminous-base-control-beta")
    
    return response.completions[0].completion

In [50]:
search_and_answer("What is the capital of France?")

' not answerable'

In [51]:
search_and_answer("What can I do with the community edition?")

' The community edition of IBM ILOG CPLEX Optimization Studio allows users to fully test the software and run models with up to 1,000 variables and 1,000 constraints.'